In [ ]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import keras
from keras import backend as K

# from keras.models import load_model
from keras.utils import np_utils
# from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten  
# from keras.layers import concatenate
from keras.layers import *
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,Adam

In [ ]:
PATH = os.getcwd()

### Define data path  ###
data_path = PATH + '/CT'
data_dir_list = os.listdir(data_path)


In [ ]:
### Pre-Processing ###
img_rows=224
img_cols=224
num_channel=1
num_epoch=20

# Define the number of classes
num_classes = 2
labels_name={'Covid':0,'non-Covid':1}

img_data_list=[]
labels_list = []
moment_list = []

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    for img in img_list:
        ## Image processing using openCV ##
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_gray=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize_gray = cv2.resize(input_img_gray,(224,224))
#         #ret,input_img = cv2.threshold(input_img,100,255,cv2.THRESH_BINARY)
#         ret,input_img = cv2.threshold(input_img, 120,255, cv2.THRESH_TOZERO)
#         #ret,input_img = cv2.threshold(input_img,100,255, cv2.THRESH_TOZERO_INV)
#         #ret,input_img = cv2.threshold(input_img, 150,255,cv2.THRESH_TRUNC)
        input_img_resize=cv2.resize(input_img,(224,224))
        moments = cv2.moments(input_img_resize_gray)
        huMoments = cv2.HuMoments(moments)   
        moment_list.append(huMoments)
        img_data_list.append(input_img_resize)
        labels_list.append(label)  


In [ ]:
### Image normalization ###
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
moment_data = np.array(moment_list)
moment_data = moment_data.astype('float32')
moment_data /= 255
print (img_data.shape)


In [ ]:
### Converting label to categorical label
from keras.utils.np_utils import to_categorical
labels = np.array(labels_list)
# print the count of number of samples for different classes
print(np.unique(labels,return_counts=True))
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)
Y.shape

In [ ]:
### Channel expansion ### *For images using RGB only
moment_data= np.expand_dims(moment_data, axis=3) 

In [ ]:
### Expanding colour channel ### *For images using Gray only

if num_channel==1:
    if K.common.image_dim_ordering()=='th':
        img_data= np.expand_dims(img_data, axis=1) 
            moment_data= np.expand_dims(moment_data, axis=1) 
        print (img_data.shape)
            print (moment_data.shape)
    else:
        img_data= np.expand_dims(img_data, axis=3) 
            moment_data= np.expand_dims(moment_data, axis=3) 
        print (img_data.shape)
            print (moment_data.shape)

else:
    if K.image_dim_ordering()=='th':
        img_data=np.rollaxis(img_data,3,1)
        print (img_data.shape)

In [ ]:
### Data Splitting ###
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
x2,y2 = shuffle(moment_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.1, random_state=2)
X_test.shape

In [ ]:
## Custom CNN configuration ##
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.adam(lr=0.0004 , decay= 1e-5) ,metrics=['accuracy'])
model.summary()

#model.save('model/CNN_Classifier.h5')

In [ ]:
## Training parameters saving ##
import pandas as pd
hist_df = pd.DataFrame(hist.history) 

hist_csv_file = 'history/CNN_history2.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
history = pd.read_csv("history/CNN_history2.csv")
val_loss = history['val_loss'].to_numpy()
val_acc = history['val_accuracy'].to_numpy()
loss = history['loss'].to_numpy()
acc = history['accuracy'].to_numpy()


In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(acc)
plt.plot(loss)
plt.plot(val_acc)
plt.plot(val_loss)
plt.title('Training Accuracy', fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('Accuracy (%)', fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel('Epoch',fontsize=20)
plt.legend(['Train_acc', 'Train_loss', 'Val_acc', 'Val_loss'], loc='best', fontsize=22)
plt.gcf().set_size_inches(12,13) 
plt.savefig('# Acc vs Loss graph path #')
plt.show()

In [ ]:
## Loading saved trained CNN model ##
#model = keras.models.load_model('model/CNN_Classifier.h5')

#### Testing phase (Without moment invariant) ####
PATH = os.getcwd()
data_path = PATH + '/Testing_Set'
data_dir_list = os.listdir(data_path)

import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix, roc_curve

test_list = []
y_pred = []
labels_name ={'Covid':0,'non-Covid':1}

#ret,thresh3 = cv2.threshold(input_img,200,255,cv2.THRESH_TRUNC)

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
#         input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
#         #ret,input_img = cv2.threshold(input_img,100,255,cv2.THRESH_BINARY)
#         ret,input_img = cv2.threshold(input_img, 120,255, cv2.THRESH_TOZERO)
        input_img_resize=cv2.resize(input_img,(224,224))
        test_list.append(input_img_resize)
        y_pred.append(label)

test_image = np.array(test_list)
test_image = test_image.astype('float32')
test_image /= 255

## For gray images only ##
# if num_channel==1:
#     if K.common.image_dim_ordering()=='th':
#         test_image= np.expand_dims(test_image, axis=1) 
#         print (img_data.shape)
#     else:
#         test_image= np.expand_dims(test_image, axis=3) 
#         #print (test_image.shape)

# else:
#     if K.image_dim_ordering()=='th':
#         test_image=np.rollaxis(test_image,3,1)
#         print (test_image.shape)
#     else:
#         test_image= np.expand_dims(test_image, axis=0)
#         print (test_image.shape)


predict_list = []
predict_1 = model.predict_proba(test_image) ## Probability prediction ##


from sklearn.metrics import auc
import pandas as pd

### ROC Curves ###
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_pred, predict_1[:,1])
auc_keras = auc(fpr_keras, tpr_keras)
optimum = thresholds_keras[np.argmax(tpr_keras - fpr_keras)]
print('Optimal cut off point = ',optimum)
print('AUC = ', auc_keras)

gmeans = np.sqrt(tpr_keras * (1-fpr_keras))
ix = np.argmax(gmeans)
optimal = thresholds_keras[ix]
print('Optimal cut off point = ',thresholds_keras[ix]) ## Optimal point ##

x = optimal # Best => x = 0.98892

# for j in range(0,1):
#     for i in range(0, len(predict_1)):
#         if predict_1[i][1] >= x:
#             predict_list.append(1)
#         else:
#             predict_list.append(0)
#     print(confusion_matrix(y_pred, predict_list), 'x =',"%.5f" % x, '\n')




cf_matrix= confusion_matrix(y_pred, predict_list)
# cf = cf_matrix
# cm_sum = np.sum(cf_matrix, axis=1, keepdims=True) 
# cm_perc = cf_matrix / cm_sum.astype(float) * 100
# annot = np.empty_like(cf_matrix).astype(str)
# nrows, ncols = cf_matrix.shape
# for i in range(nrows):
#     for j in range(ncols):
#         c = cf_matrix[i, j]
#         p = cm_perc[i, j]
#         if i == j:
#             s = cm_sum[i]
#             annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
#         elif c == 0:
#             annot[i, j] = ''
#         else:
#             annot[i, j] = '%.1f%%\n%d' % (p, c)
# cf_matrix = pd.DataFrame(cf_matrix, index=labels, columns=labels)
# print(cm_perc)
tp = cf_matrix[0][0]
fn = cf_matrix[0][1]
fp = cf_matrix[1][0]
tn = cf_matrix[1][1]

TrueP = '{0:.2%}'.format(tp/(tp+fn))
FalseN = '{0:.2%}'.format(fn/(tp+fn))
FalseP = '{0:.2%}'.format(fp/(fp+tn))
TrueN = '{0:.2%}'.format(tn/(fp+tn))
print(TrueP, FalseN, FalseP, TrueN)

xy_Labels = ['Covid', 'non-Covid']
group_names = ['True Positive', 'False Negative', 'False Positive','True Negative']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
group_percentage = [TrueP, FalseN, FalseP, TrueN]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentage)]
labels = np.asarray(labels).reshape(2,2)


### Confusion matrix ###
# plt.title('(CNN), Threshold = '+str("{:.4f}".format(optimal)), fontsize = 20)
plt.title('Confusion Matrix (Custom CNN)',fontsize = 18)
sns.heatmap(cf_matrix, fmt="", annot=labels, annot_kws={'size':18} ,xticklabels=xy_Labels, yticklabels=xy_Labels, cmap='Blues')
plt.xlabel('Predicted label', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('True label', fontsize = 15) # y-axis label with fontsize 15
plt.savefig('## Path saving CF')
plt.show()

### ROC graph ###
plt.figure(figsize=(6,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.scatter(fpr_keras[ix],tpr_keras[ix], marker='o', color='black', label='Optimal Threshold = {:.4f})'.format(optimal))
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.title('ROC (CNN)',fontsize=22)
plt.legend(loc='best', fontsize=15)
plt.savefig('## Path saving ROC Curves')
plt.show()




#model.save('model/CNN_Threshold_1.h5')